In [2]:
import numpy as np

#calculation by using Hilbert_Matrix
#Required n,H_M, b, x_T
import numpy as np
import fractions
#Hilbert Matrix is for precision data in matrix
class generatorHb:
    def __init__(self, req_n):
        self.n = req_n
        self.hilbert_matrix = self.Hilbert_Matrix()
#Function for generate hilbert matrix
    def Hilbert_Matrix(self):
        H_M = np.zeros((self.n, self.n), dtype=float)
        for i in range(1, self.n + 1):
            for j in range(1, self.n + 1):
                H_M[i - 1, j - 1] = 1 / (i + j - 1)

        #print(H_M)
        return H_M



In [3]:




class Gauss_Elimination:
    #initial
    def __init__(self, Matrix, b):
        self.M = Matrix
        self.b = b
        self.n = len(b)
        self.x = np.ones(self.n)#change to all one
        self.x_head=np.zeros(self.n)#build initial matrix
    #combine Hilbert Matrix with b matrix
    def test_M(self):
        matrix = np.concatenate((self.M, np.reshape(self.b, (self.n, 1))), axis=1)
        return matrix
    #forward substitution
    def forward_elimination(self):
        A_inv = np.eye(self.n)
        test_matrix = self.test_M()
        for k in range(self.n - 1):
            pivot = test_matrix[k, k]
            if pivot == 0:
                continue
            for i in range(k + 1, self.n):
                factor = test_matrix[i, k] / pivot
                test_matrix[i, k:] -= factor * test_matrix[k, k:]
                A_inv[i, k] = factor

        return test_matrix, A_inv

    def backward_substitution(self):
        test_matrix,  _= self.forward_elimination()
        for j in range(self.n - 1, -1, -1):
            pivot = test_matrix[j, j]
            if pivot == 0:
                continue
            self.x_head[j] = test_matrix[j, -1] / pivot
            for i in range(j - 1, -1, -1):
                test_matrix[i, -1] -= test_matrix[i, j] * self.x_head[j]

        return self.x_head

    def gauss_elimination_calculation(self):
        test_matrix, A_inv = self.forward_elimination()
        self.x_head = self.backward_substitution()

        return test_matrix, A_inv, self.x_head

    def residual_calculation_norm(self):
        r=[]
        #dot product of hilbert matrix and x_head
        Mdot=np.dot(self.M,np.transpose(self.x_head))
        # residual= b-Hx_head
        for i in range(self.n):
                r_value=self.b[i]-Mdot[i]
                r.append(r_value)
        #residual inf norm
        r_norm=np.linalg.norm(r,np.inf)
        return r, r_norm

    def error_norm(self):

        #array: x_delta=x_true-x_head
        x_delta=self.x-self.x_head
        #X_delta norm inf
        X_norm=np.linalg.norm(x_delta,np.inf)
        #x_true norm inf
        x_true_norm=np.linalg.norm(x_true,np.inf)
        err_present=X_norm/x_true_norm

        return x_delta,X_norm,err_present

    def condition_H(self):
        condition=np.linalg.cond(self.M, np.inf)

        return condition




In [8]:
#start position 2 =>n=1
n=2
relative_err=0

n_list=[]
r_norm_list=[]
cond_H_list=[]
while  relative_err<=1:
    #Generate Hilbert_matrix
    hilbert_matrix_calculation = generatorHb(n)
    Matrix = hilbert_matrix_calculation.hilbert_matrix
    #x_true matrix are all ones
    x_true=np.transpose(np.ones(n))
    #b=Hx_true
    b=np.dot(Matrix,x_true)
    #perform Gauss_Elimination with Hilbert matrix and b
    test = Gauss_Elimination(Matrix, b)
    test_matrix, A_inv, x_head = test.gauss_elimination_calculation()
    #residual calculation
    r,r_norm=test.residual_calculation_norm()
    # x_delta,x_delta_norm, and err_percentage=x_delta_norm/ x_true
    x_delta,x_norm,err_percentage=test.error_norm()
    #Condition of Hilbert Matrix for each n
    H_condition=test.condition_H()
    #when err_percentage>relative_err, loop break
    relative_err=err_percentage
    print(f'n:{n},\t\terror_percentage={relative_err:.4%},\n x_head={x_head},\nx_delta={x_delta},\nr={r},\n')
    #append to the array and demonstrate as table

    n_list.append(n)
    r_norm_list.append(float(r_norm))
    cond_H_list.append(H_condition)
    n+=1

print(f'n\t\tr_norm\t\t\t\tCond(H)\t\t')
for i in range(len(n_list)):
    print(f'{n_list[i]}\t\t{r_norm_list[i]:.4e}\t\t\t\t{cond_H_list[i]:.4f}')

n:2,		error_percentage=0.0000%,
 x_head=[1. 1.],
x_delta=[-4.44089210e-16  6.66133815e-16],
r=[0.0, 0.0],

n:3,		error_percentage=0.0000%,
 x_head=[1. 1. 1.],
x_delta=[ 2.10942375e-15 -1.22124533e-14  1.25455202e-14],
r=[2.220446049250313e-16, 0.0, 0.0],

n:4,		error_percentage=0.0000%,
 x_head=[1. 1. 1. 1.],
x_delta=[-4.21884749e-15  5.11812814e-14 -1.21458399e-13  7.77156117e-14],
r=[0.0, 0.0, 1.1102230246251565e-16, 0.0],

n:5,		error_percentage=0.0000%,
 x_head=[1. 1. 1. 1. 1.],
x_delta=[ 3.45279361e-14 -5.23359134e-13  1.93722816e-12 -2.59126054e-12
  1.14752652e-12],
r=[0.0, 2.220446049250313e-16, 0.0, 1.1102230246251565e-16, 1.1102230246251565e-16],

n:6,		error_percentage=0.0000%,
 x_head=[1. 1. 1. 1. 1. 1.],
x_delta=[ 4.64517314e-13 -1.27049482e-11  8.35800318e-11 -2.13027596e-10
  2.31531239e-10 -9.01261288e-11],
r=[0.0, 2.220446049250313e-16, 0.0, 0.0, 0.0, 0.0],

n:7,		error_percentage=0.0000%,
 x_head=[1.         1.         1.         1.         0.99999999 1.00000001
 1.  